# 提示工程（Prompt engineering）一些技巧

提示工程是人工智慧中的一個概念，特別是自然語言處理 (NLP) 領域。 在提示工程中，透過用戶輸入任務的描述。引導與干涉模型產出正確的結果。這種以提示為基礎的學習（prompt-based learning）」是運用大型語言模型的主要方式。提示工程有多種技巧 (https://github.com/dair-ai/Prompt-Engineering-Guide) 以下示範幾種典型的技巧。

In [1]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# 載入環境變數
load_dotenv()

# 呼叫 OpenAI ChatComplete API 
def call_openai_api(prompt, temperature=0, max_token=100):
    # 設定呼叫 Azure OpenAI Service API 所需連線資訊
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    api_key=os.getenv("AZURE_OPENAI_API_KEY") 
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
    
    client = AzureOpenAI(
        azure_endpoint=azure_endpoint, 
        api_key=api_key,  
        api_version=api_version
    )
    model = os.getenv('CHAT_DEPLOYMENT_NAME')

    response = client.chat.completions.create(
        model= model,    
        messages =  [{"role": "system", "content": "你是以正體中文回覆的機器人，不要使用 Markdown 語法回覆。"},{'role': 'user', 'content': prompt}],
        temperature = temperature,
        max_tokens=max_token)

    return (response.choices[0].message.content)

# 思維鍊 (Chain of Thought, CoT)

大型語言模型在面對一個需要數學運算與推理的問題任務，直接生成的答案往往是錯誤的，但在過去幾年研究發現，將決問題的步驟逐一拆解，搭配精心設計的提示，能夠大幅提高大型語言模型生成答案的正確性。這種引導大型語言模型思路的技巧稱為思維鍊 (Chain of Thought, COT)，以下是一些的思維鍊範例。

In [2]:
# 以零樣本學習（Zero-shot learning）的方式，GPT-4o 的答案是錯的

PROMPT_ZERO_SHOT =  "情境：五個人（A、B、C、D 和 E）在一個房間裡。"  \
                    "A 和 B 正在看電視，D 正在睡覺，B 正在吃炒麵，E 正在打乒乓球。" \
                    "突然，電話響了。B 走出房間接電話。請問 C 在做什麼？"

response = call_openai_api(PROMPT_ZERO_SHOT, temperature=0, max_token=600)

print(response)

根據情境描述，A 和 B 正在看電視，D 正在睡覺，B 正在吃炒麵，E 正在打乒乓球。並沒有提到 C 在做什麼。因此，無法確定 C 在做什麼。


以下開始使用思維鍊 (CoT) 引導模型推理

In [3]:
# 以 Few Shots 搭配思維鍊 (CoT) , 引導出正確的答案

PROMPT_FEW_SHOTS=  "情境：五個人（A、B、C、D 和 E）在一個房間裡。"  \
                    "A 和 B 正在看電視，D 正在睡覺，B 正在吃炒麵，E 正在打乒乓球。" \
                    "突然，電話響了。B 走出房間接電話。請問 C 在做什麼？" \
                    "提示:乒乓球一定要兩個人才能打"

response = call_openai_api(PROMPT_FEW_SHOTS, temperature=0, max_token=600)

print(response)

根據提示，乒乓球一定要兩個人才能打，而E正在打乒乓球，因此C應該是和E一起打乒乓球。


# 更多思維鍊 (CoT) 提示的案例




# Program-aided Language Models
PAL (Program-aided Language) Models 是思維鍊的技巧之一，是利用程式語言的數學公式來引導大型語言模型推理 (https://cobusgreyling.medium.com/pal-program-aided-large-language-models-30db3e59f796)， 原本 GPT 是大型語言模型原本並非設計出來處理數學問題，以 PAL 能夠提升數學計算能力，隨著模型版本更新，數學推理能力也逐漸增強，許多傳統代數問題 GPT-4o 已可以在 Zero-Shot 的情況下回答正確。

In [4]:
# 以零樣本學習（Zero-shot learning）處理單純代數問題 GPT-4o 已經不需要 PAL 引導就能回答

PROMPT_ZERO_SHOT =  "學校圖書館有故事書、科技書和漫畫畫共1200本，其中故事書佔60%，科技書和漫畫畫數量比是2：3，圖書館有多少本漫畫畫?"

response = call_openai_api(PROMPT_ZERO_SHOT, temperature=0, max_token=600)

print(response)

首先，我們先計算故事書的數量。故事書佔總數的60%，所以：

故事書數量 = 1200本 * 60% = 1200本 * 0.6 = 720本

接下來，我們計算科技書和漫畫畫的總數。由於故事書佔了720本，剩下的就是科技書和漫畫畫的數量：

科技書和漫畫畫的總數 = 1200本 - 720本 = 480本

根據題目，科技書和漫畫畫的數量比是2：3。假設科技書的數量是2x，漫畫畫的數量是3x，則：

2x + 3x = 480本
5x = 480本
x = 480本 / 5
x = 96本

因此，漫畫畫的數量是3x：

漫畫畫數量 = 3 * 96本 = 288本

所以，圖書館有288本漫畫畫。


In [5]:
# GPT-4o 對於小數點判斷仍有可能判斷錯誤

PROMPT_ZERO_SHOT =  "3.14 與 3.9 哪個數字比較大?"

response = call_openai_api(PROMPT_ZERO_SHOT, temperature=0, max_token=600)

print(response)

3.14 比 3.9 大。


In [15]:
# 以 PAL 方式引導 GPT-4o 比較福點數大小，正確計算機會大幅增加

PROMPT_FEW_SHOTS  =  "比較兩個浮點數 a 與 b 大小時，可利用 c = a -b 的結果來判斷，若 c > 0 則 a 比 b 大， 若 c < 0 則 b 比 a 大 " \
                     "請問 3.14 與 3.9 哪個數字比較大?"

response = call_openai_api(PROMPT_FEW_SHOTS, temperature=0, max_token=600)

print(response)

根據你提供的方法，我們可以計算 c = 3.14 - 3.9。

c = 3.14 - 3.9 = -0.76

因為 c < 0，所以 3.9 比 3.14 大。


In [17]:
# 不了解一條鐵路需要兩條鐵軌，GPT-4o 會回答錯誤

PROMPT_ZERO_SHOT =  "一段長100公尺的鐵路，用10公尺長的鐵軌鋪，要多少根鐵軌？"

response = call_openai_api(PROMPT_ZERO_SHOT, temperature=0, max_token=600)

print(response)

要鋪設一段長100公尺的鐵路，每根鐵軌長10公尺，因此需要的鐵軌數量可以通過以下計算得出：

100公尺 ÷ 10公尺/根 = 10根

所以，需要10根鐵軌。


In [16]:
# 以 PAL 方式引導 GPT-4o 正確計算

PROMPT_FEW_SHOTS =  "一段長100公尺的鐵路，用10公尺長的鐵軌鋪，要多少根鐵軌？" \
                    "計算方式: 一條鐵路需要兩條鐵軌 1 railway = 2 rails" \
                    " total_rails_length = 1 x 2 100  = 200" \
                    " single_rail_length = 10" \
                    " total_rails = total_rails_length / single_rail_length = 200 / 10 = 20 ，所以需要 20 根鐵軌" \
                    "一段長800公尺的鐵路，用10公尺長的鐵軌鋪，要多少根鐵軌？"
response = call_openai_api (PROMPT_FEW_SHOTS, temperature=0, max_token=600)

print(response)

我們可以使用相同的計算方式來解決這個問題。

1. 一條鐵路需要兩條鐵軌。
2. 總鐵軌長度 = 鐵路長度 x 2
3. 單根鐵軌長度 = 10 公尺
4. 總鐵軌數量 = 總鐵軌長度 / 單根鐵軌長度

現在我們來計算：

1. 鐵路長度 = 800 公尺
2. 總鐵軌長度 = 800 公尺 x 2 = 1600 公尺
3. 單根鐵軌長度 = 10 公尺
4. 總鐵軌數量 = 1600 公尺 / 10 公尺 = 160 根

所以，一段長 800 公尺的鐵路，用 10 公尺長的鐵軌鋪，需要 160 根鐵軌。


# 常識推論 (Commonsense Reasoning)

2019 年使用 OpenAI GPT 語言模型架構持續增加人類常識知識庫加以訓練 https://www.quantamagazine.org/common-sense-comes-to-computers-20200430/ ，幾乎可達到接近人類水準的常識性推理。近年來大型語言模型持續補充了更多高品質的知識，幾乎已經具備人類水準常識推論能力。以下的經典問題早期 GPT-3.5 Turbo 版本都無法正確回應目前，而目前 GPT-4o 都可以做出正確的常識推論，尚在尋找 GPT-4o 無法做出正確的常識推論的問題。

In [45]:
# 地理空間常識問題 - GPT-3.5 得到錯誤的答案，GPT-4o 已經可以得到正確的答案

PROMPT = "3月5日上午九點行程巴黎凡爾賽宮，下午參觀巴黎聖母院，晚上八點參加夜遊塞納河。" \
"3月6上午八點參觀雪梨歌劇院，下午參觀雪梨港灣大橋，晚上八點參加夜遊雪梨港邊海景餐廳。" \
 "這個旅遊行程你可能會是哪一種情緒 a. 生氣 b. 無聊 c. 快樂 d. 輕鬆，請問答案是哪一個?"
response = call_openai_api(PROMPT,temperature=0 , max_token=800)
print(response)

這個旅遊行程安排得非常緊湊，並且涉及兩個不同的城市（巴黎和雪梨），這在現實中是不太可能實現的，因為這兩個城市相距甚遠，需要長時間的飛行。因此，這樣的行程可能會讓人感到壓力和疲憊。

如果要選擇一種情緒，我可能會選擇 a. 生氣，因為這樣的行程安排不切實際，會讓人感到困惑和不滿。


In [32]:
# 生理常識問題 - GPT-4 得到錯誤的答案，GPT-4o 已經可以得到正確的答案

PROMPT = "小美在上午 9 點的心率為 75 次/分，晚上 7 點的血壓為 120/80。她在晚上 11 點去世。她中午時刻還活著嗎？ "
response = call_openai_api(PROMPT,temperature=0 , max_token=800)

print(response)

根據提供的信息，小美在上午 9 點的心率為 75 次/分，晚上 7 點的血壓為 120/80，這些數據顯示她在這些時間點都是活著的。她在晚上 11 點去世，這意味著她在當天晚上 11 點之前都是活著的。

因此，可以合理地推斷，小美在前天中午還是活著的。


In [27]:
# 生理常識問題 - GPT-3.5 得到錯誤的答案，GPT-4o 已經可以得到正確的答案

PROMPT = "游泳池加熱至攝氏 100 度，水獺與人類游泳選手在這個游泳池比賽 100 公尺游泳，誰會勝出? "
response = call_openai_api(PROMPT,temperature=0 , max_token=800)

print(response)

如果游泳池的水被加热到摄氏100度（即沸点），那么在这种极端条件下，无论是水獭还是人类游泳选手，都无法在这样的环境中游泳。摄氏100度的水会导致严重的烫伤，甚至可能致命。因此，在这种情况下，比赛根本无法进行，因为任何进入这种高温水中的生物都会受到严重伤害。

为了进行公平和安全的比赛，游泳池的水温通常会保持在一个适宜的范围内，通常在摄氏25到28度之间。在这种正常的水温下，人类游泳选手和水獭的比赛结果可能会有所不同，但在摄氏100度的水中，比赛是不可能进行的。


In [29]:
# 歷史時事常識問題 - GPT-3.5 得到錯誤的答案，GPT-4o 已經可以得到正確的答案

PROMPT = "對於一個在 1968 年進行自然考試的學生而言，在考卷中有一題 '太陽系有幾大行星？' 這題正確答案是什麼? "
response = call_openai_api(PROMPT,temperature=0 , max_token=800)

print(response)

在1968年，太陽系被認為有九大行星。這九大行星依次是：水星、金星、地球、火星、木星、土星、天王星、海王星和冥王星。因此，對於1968年的學生來說，這題的正確答案是九大行星。

需要注意的是，2006年國際天文學聯合會（IAU）重新定義了行星的標準，將冥王星重新分類為矮行星。因此，根據現代標準，太陽系只有八大行星，但這不適用於1968年的情況。


In [30]:
# 歷史時事常識問題 - GPT-3.5 得到錯誤的答案，GPT-4o 已經可以得到正確的答案

PROMPT = "對於一個在 1890 年進行自然考試的學生而言，在考卷中有一題 '太陽系有幾大行星？' 這題正確答案是什麼? "
response = call_openai_api(PROMPT,temperature=0 , max_token=800)

print(response)

在1890年，根據當時的天文知識，太陽系被認為有8顆大行星。這些行星是：

1. 水星 (Mercury)
2. 金星 (Venus)
3. 地球 (Earth)
4. 火星 (Mars)
5. 木星 (Jupiter)
6. 土星 (Saturn)
7. 天王星 (Uranus)
8. 海王星 (Neptune)

冥王星（Pluto）是在1930年被發現的，因此在1890年並不被認為是太陽系的一部分。即使在冥王星被發現後，它在2006年被重新分類為矮行星。因此，1890年的正確答案是8顆大行星。
